# Virtual Home Planner using LAPKT

The following provides a complete instruction on generating actions given Virtualhome environment intial setting defined using a pddl file. Please see 'example_problem.pddl' to know how these problem files are defined.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1_FMnK6rmCjVCtQbj18qbuhxJGrIvA7Up?usp=sharing)

# Setup PDDLMultiAgent Planner Directory

In [1]:
!git -c core.symlinks=true clone --recurse-submodules https://github.com/xavierpuigf/PDDLMultiAgent
%cd PDDLMultiAgent

Cloning into 'PDDLMultiAgent'...
remote: Enumerating objects: 197, done.
remote: Total 197 (delta 0), reused 0 (delta 0), pack-reused 197
Receiving objects: 100% (197/197), 210.27 KiB | 6.78 MiB/s, done.
Resolving deltas: 100% (96/96), done.
Submodule 'LAPKT-public' (https://github.com/LAPKT-dev/LAPKT-public.git) registered for path 'LAPKT-public'
Cloning into '/content/PDDLMultiAgent/LAPKT-public'...
remote: Enumerating objects: 11594, done.        
remote: Counting objects: 100% (1173/1173), done.        
remote: Compressing objects: 100% (331/331), done.        
remote: Total 11594 (delta 897), reused 842 (delta 842), pack-reused 10421        
Receiving objects: 100% (11594/11594), 15.03 MiB | 4.32 MiB/s, done.
Resolving deltas: 100% (6976/6976), done.
Submodule path 'LAPKT-public': checked out 'af757138d0b4d903e35e5946b3b88e8f8d64e976'
/content/PDDLMultiAgent


# Build LAPKT

In [2]:
!sudo apt-get install xutils-dev
!sudo apt install flex 
!sudo apt install bison
!python -m pip install scons

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  xutils-dev
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 239 kB of archives.
After this operation, 1,503 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 xutils-dev amd64 1:7.7+5ubuntu1 [239 kB]
Fetched 239 kB in 1s (275 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package xutils-dev.
(Reading database ... 155501 files and direc

In [3]:
%cd LAPKT-public
!scons

/content/PDDLMultiAgent/LAPKT-public
scons: Reading SConscript files ...
  File "/content/PDDLMultiAgent/LAPKT-public/SConstruct", line 31

    if src_objs is None : print "src_objs is None"

                                                 ^

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("src_objs is None")?



In [4]:
%cd ./external/libff
!make clean
!make depend
!make

/content/PDDLMultiAgent/LAPKT-public/external/libff
rm -f *.o *.bak *~ *% core *_pure_p9_c0_400.o.warnings test_lib \
        \#*\#  scan-fct_pddl.tab.c scan-ops_pddl.tab.c scan-probname.tab.c lex.fct_pddl.c lex.ops_pddl.c 
makedepend -- main.c memory.c output.c parse.c expressions.c inst_pre.c inst_easy.c inst_hard.c inst_final.c relax.c search.c utility.c  libff.c memory.c output.c parse.c expressions.c inst_pre.c inst_easy.c inst_hard.c inst_final.c relax.c search.c utility.c scan-fct_pddl.tab.c scan-ops_pddl.tab.c scan-probname.tab.c lex.fct_pddl.c lex.ops_pddl.c  test_main.cxx
makedepend: warning:  main.c (reading /usr/include/stdlib.h, line 25): cannot find include file "bits/libc-header-start.h"
	not in /usr/include/bits/libc-header-start.h
makedepend: warning:  main.c (reading /usr/include/stdlib.h, line 31): cannot find include file "stddef.h"
	not in /usr/include/stddef.h
makedepend: warning:  main.c (reading /usr/include/stdlib.h, line 55): cannot find include file "bits/flo

In [5]:
%cd ../..

/content/PDDLMultiAgent/LAPKT-public


In [6]:
%cd ./planners/siw_plus-then-bfs_f-ffparser
!scons

/content/PDDLMultiAgent/LAPKT-public/planners/siw_plus-then-bfs_f-ffparser
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
g++ -o src/main.o -c -Wall -std=c++11 -O3 -DNDEBUG -I/content/PDDLMultiAgent/LAPKT-public/src -I/content/PDDLMultiAgent/LAPKT-public/include -I/content/PDDLMultiAgent/LAPKT-public/interfaces/agnostic -I/content/PDDLMultiAgent/LAPKT-public/interfaces/ff-wrapped -I/content/PDDLMultiAgent/LAPKT-public/include -I/content/PDDLMultiAgent/LAPKT-public/external -I/content/PDDLMultiAgent/LAPKT-public/interfaces/agnostic -I/content/PDDLMultiAgent/LAPKT-public/interfaces/ff-wrapped -Isrc src/main.cxx
g++ -o /content/PDDLMultiAgent/LAPKT-public/src/bit_array.o -c -Wall -std=c++11 -O3 -DNDEBUG -I/content/PDDLMultiAgent/LAPKT-public/src -I/content/PDDLMultiAgent/LAPKT-public/include -I/content/PDDLMultiAgent/LAPKT-public/interfaces/agnostic -I/content/PDDLMultiAgent/LAPKT-public/interfaces/ff-wrapped -I/content/PDDLMultiAgent

In [7]:
%cd ../../external/libff
!make clean
!make depend
!make

/content/PDDLMultiAgent/LAPKT-public/external/libff
rm -f *.o *.bak *~ *% core *_pure_p9_c0_400.o.warnings test_lib \
        \#*\#  scan-fct_pddl.tab.c scan-ops_pddl.tab.c scan-probname.tab.c lex.fct_pddl.c lex.ops_pddl.c 
makedepend -- main.c memory.c output.c parse.c expressions.c inst_pre.c inst_easy.c inst_hard.c inst_final.c relax.c search.c utility.c  libff.c memory.c output.c parse.c expressions.c inst_pre.c inst_easy.c inst_hard.c inst_final.c relax.c search.c utility.c scan-fct_pddl.tab.c scan-ops_pddl.tab.c scan-probname.tab.c lex.fct_pddl.c lex.ops_pddl.c  test_main.cxx
makedepend: warning:  main.c (reading /usr/include/stdlib.h, line 25): cannot find include file "bits/libc-header-start.h"
	not in /usr/include/bits/libc-header-start.h
makedepend: warning:  main.c (reading /usr/include/stdlib.h, line 31): cannot find include file "stddef.h"
	not in /usr/include/stddef.h
makedepend: warning:  main.c (reading /usr/include/stdlib.h, line 55): cannot find include file "bits/flo

# Sample Run a planner

In [8]:
root_dir = "/content/PDDLMultiAgent"
%cd $root_dir

/content/PDDLMultiAgent


## Implement customize siw-then-bfsf planner wrapper

The planner wrapper from the repo does not work as it depends on a symlink on github. 

In [9]:
import subprocess
import os
import pdb
import utils_env_parser
import urllib.request as urllibreq
import urllib
import argparse
import json, sys
import time

In [10]:
planner_pth = os.path.join(root_dir, 'LAPKT-public', 'planners', 'siw_plus-then-bfs_f-ffparser' ,'siw-then-bfsf')

def convert_to_virtualhome_program(action_list):
    action_list = [x[1:-2].split() for x in action_list]
    action_list_str = []
    for action_instr in action_list:
        if len(action_instr) == 0:
            continue
        action_item = '[{}]'.format(action_instr[0].upper())
        if len(action_instr) > 1:
            objects_item = ['<{}> ({})'.format('_'.join(l.split('_')[:-1]), l.split('_')[-1]) for l in action_instr[1:]]
        action_list_str.append('{} {}'.format(action_item, ' '.join(objects_item)))
    return action_list_str

def planner(domain_name, problem_name, file_out, planner_dir = planner_pth):
    start = time.time()
    stats_dir = '/'.join(file_out.split('/')[:-3]) + '/stats/' 
    file_stats = stats_dir + file_out.split('/')[-1] + '_stats.txt'
    os.makedirs(stats_dir, exist_ok=True)
    with open(file_stats, 'w+') as outfile:
        proc = subprocess.call([planner_pth, '--domain', domain_name, '--problem', problem_name, '--output', file_out], stdout=outfile)
    with open(file_out, 'r') as f:
        program = f.readlines()
        if len(program) > 0 and len(program[-1]) == 0:
            program = program[:-1]
    program = [x+'\n' for x in convert_to_virtualhome_program(program)]
    if len(program) > 0:
        with open(file_out, 'w+') as f:
            f.writelines(program)
        print("Successfully executed the planner.")
    else:
        os.remove(file_stats)
        os.remove(file_out)
        print("Planner has no output.")
    end = time.time()
    print("Time used: {}".format(end-start))


### Sample run
Replace example_problem/example_problem.pddl with the following content.


(define (problem setuptable1)
(:domain virtualhome)
(:objects
    plate_1, plate_2, plate_3, table_4 drawer_5 - object
    kitchen_6 livingroom_7 - room
    char_0 - character
    char_8 - character
)
(:init
    (= (objects_grabbed) 0)
    (container drawer_5)
    (surface table_4)
    (grabable plate_1)
    (grabable plate_2)
    (grabable plate_3)
    (inside plate_1 drawer_5)
    (inside plate_2 drawer_5)
    (inside plate_3 drawer_5)
    (inside drawer_5 kitchen_6)
    (inside table_4 livingroom_7)
    (inside char_0 livingroom_7)
    (inside char_8 kitchen_6)
)
(:goal
    (and (ontop plate_1 table_4) (ontop plate_3 table_4) (inside char_8 livingroom_7) (inside char_0 kitchen_6))
)
)

In [ ]:
#! ./LAPKT-public/planners/siw_plus-then-bfs_f-ffparser/siw-then-bfsf --domain domain.pddl --problem example_problem/example_problem.pddl --output file_out

In [ ]:
domain_name='./domain.pddl'
problem_name='./example_problem/example_problem.pddl'
output_dir = "./example_out"
planner(domain_name=domain_name, problem_name=problem_name, file_out=output_dir)

Successfully executed the planner.
Time used: 0.014673233032226562


In [ ]:
#printout result
f = open('./example_out')
for line in f:
    print(line)

[WALK] <> (KITCHEN) <> (CHAR1)

[WALK] <> (DRAWER) <> (CHAR1)

[OPEN] <> (DRAWER) <> (CHAR1)

[WALK] <> (PLATE3) <> (CHAR1)

[GRAB] <> (PLATE3) <> (CHAR1)

[WALK] <> (LIVINGROOM) <> (CHAR1)

[WALK] <> (TABLE) <> (CHAR1)

[PUT] <> (PLATE3) <> (TABLE) <> (CHAR1)

[WALK] <> (KITCHEN) <> (CHAR1)

[WALK] <> (DRAWER) <> (CHAR1)

[WALK] <> (PLATE1) <> (CHAR1)

[GRAB] <> (PLATE1) <> (CHAR1)

[WALK] <> (LIVINGROOM) <> (CHAR1)

[WALK] <> (TABLE) <> (CHAR1)

[PUT] <> (PLATE1) <> (TABLE) <> (CHAR1)



# Customized under Here

In [11]:
import json
## add tasks
## remember to upload goals.json


tasks = open('/content/PDDLMultiAgent/goals.json')
tasks = json.load(tasks)

In [12]:
tasks

{'(and  (ontop bananas_316 nightstand_106) (inside clothespants_141 clothespile_150) (grabbed character_1 hanger_119) (grabbed character_2 clothespants_379))': [' (ontop bananas_316 nightstand_106)',
  ' (inside clothespants_141 clothespile_150)',
  '(grabbed character_1 hanger_119)',
  '(grabbed character_2 clothespants_379)'],
 '(and  (ontop bananas_316 plate_274) (inside cereal_335 clothespile_413) (grabbed character_1 clothespile_150) (grabbed character_2 hanger_148))': [' (ontop bananas_316 plate_274)',
  ' (inside cereal_335 clothespile_413)',
  '(grabbed character_1 clothespile_150)',
  '(grabbed character_2 hanger_148)'],
 '(and  (ontop bananas_440 bench_232) (inside book_435 clothespile_169) (grabbed character_1 clothespile_169) (grabbed character_2 plate_274))': [' (ontop bananas_440 bench_232)',
  ' (inside book_435 clothespile_169)',
  '(grabbed character_1 clothespile_169)',
  '(grabbed character_2 plate_274)'],
 '(and  (ontop bananas_440 coffeetable_113) (inside hanger_11

In [13]:
!mkdir -p sample_env/input_envs
!mkdir -p sample_env/out_problems
#place simplified_sample.json under input_envs

In [14]:
!python env_parser.py --folder_env sample_env/input_envs --folder_out sample_env/out_problems

In [15]:
list(tasks.keys())[40]

'(and  (ontop hanger_380 bench_232) (inside hanger_117 clothespile_400) (grabbed character_1 dishbowl_318) (grabbed character_2 cutleryfork_277))'

In [16]:
tasks.keys()

dict_keys(['(and  (ontop hanger_138 nightstand_106) (inside clothespants_383 clothespile_169) (grabbed character_1 hanger_119) (grabbed character_2 dishbowl_289))', '(and  (ontop hanger_117 nightstand_106) (inside clothespants_379 closet_375) (grabbed character_1 dishbowl_289) (grabbed character_2 clothespants_141))', '(and  (ontop clothespile_165 bathroomcabinet_49) (inside hanger_148 kitchencabinet_235) (grabbed character_1 bellpepper_322) (grabbed character_2 hanger_119))', '(and  (ontop clothespile_165 kitchencabinet_235) (inside towel_54 bathroomcabinet_49) (grabbed character_1 bellpepper_322) (grabbed character_2 plate_274))', '(and  (ontop clothespants_379 bathroomcabinet_49) (inside clothesshirt_131 clothespile_398) (grabbed character_1 clothespile_398) (grabbed character_2 hanger_138))', '(and  (ontop clothespile_400 nightstand_106) (inside clothespants_141 kitchencabinet_235) (grabbed character_1 towel_54) (grabbed character_2 dishbowl_318))', '(and  (ontop clothespile_150 be

In [ ]:
# content = ""
# outputfile = "./sample_env/out_problems/file_0.pddl"
# goal = '(and (inside clothespants_387 clothespile_413) (inside plate_195 kitchencabinet_235) (grabbed character_1 towel_68) (grabbed character_2 cereal_335))'
# flag = False

# f = open('/content/PDDLMultiAgent/sample_env/out_problems/file_0.pddl', 'r')
# for line in f:
#     #print(line)
#     content += line if not flag else ""
#     if line.startswith('(:goal'):
#         flag = True 
#     if flag and line.startswith(')'):
#         flag = False
#         content += goal + '\n'
#         content += ')\n'
        
# wf = open(outputfile, 'w')
# wf.write(content)
# wf.close()
# domain_name='./domain.pddl'
# problem_name='./sample_env/out_problems/file_0.pddl'
# output_dir = "./example_out"
# ! ./LAPKT-public/planners/siw_plus-then-bfs_f-ffparser/siw-then-bfsf --domain $domain_name --problem $problem_name --output $output_dir


unknown constant CLOTHESPANTS_387 in literal INSIDE. check input files



In [ ]:
# steps = []
# print('Goals: ', goal)
# f = open('./example_out')
# for line in f:
#     steps.append(line)

# steps

Goals:  (and (inside clothespants_387 clothespile_413) (inside plate_195 kitchencabinet_235) (grabbed character_1 towel_68) (grabbed character_2 cereal_335))


['(WALK BATHROOM_11 CHARACTER_1)\n',
 '(WALK TOWEL_68 CHARACTER_1)\n',
 '(GRAB TOWEL_68 CHARACTER_1)\n',
 '(WALK KITCHEN_207 CHARACTER_2)\n',
 '(WALK CEREAL_335 CHARACTER_2)\n',
 '(GRAB CEREAL_335 CHARACTER_2)\n',
 '(WALK KITCHENCABINET_235 CHARACTER_2)\n',
 '(OPEN KITCHENCABINET_235 CHARACTER_2)\n',
 '(WALK PLATE_195 CHARACTER_2)\n',
 '(GRAB PLATE_195 CHARACTER_2)\n',
 '(PUTIN PLATE_195 KITCHENCABINET_235 CHARACTER_2)\n',
 '(WALK LIVINGROOM_336 CHARACTER_1)\n',
 '(WALK CLOTHESPANTS_387 CHARACTER_1)\n',
 '(GRAB CLOTHESPANTS_387 CHARACTER_1)\n',
 '(WALK CLOTHESPILE_413 CHARACTER_1)\n',
 '(OPEN CLOTHESPILE_413 CHARACTER_1)\n',
 '(PUTIN CLOTHESPANTS_387 CLOTHESPILE_413 CHARACTER_1)\n']

In [17]:
count = 0
method = {}
for key in tasks.keys():


    content = ""
    outputfile = "./sample_env/out_problems/file_0.pddl"
    goal = key
    count+=1
    print(count, key)
    flag = False

    f = open('/content/PDDLMultiAgent/sample_env/out_problems/file_0.pddl', 'r')
    for line in f:
      #print(line)
      content += line if not flag else ""
      if line.startswith('(:goal'):
        flag = True 
      if flag and line.startswith(')'):
        flag = False
        content += goal + '\n'
        content += ')\n'
        
    wf = open(outputfile, 'w')
    wf.write(content)
    wf.close()
    domain_name='./domain.pddl'
    problem_name='./sample_env/out_problems/file_0.pddl'
    output_dir = "./example_out"

    ! ./LAPKT-public/planners/siw_plus-then-bfs_f-ffparser/siw-then-bfsf --domain $domain_name --problem $problem_name --output $output_dir



    steps = []
    f = open('./example_out')
    for line in f:
      steps.append(line)

    method[key] = steps


    



流式输出内容被截断，只能显示最后 5000 行内容。
{4/2/2}:IW(1) -> [2][3][4][5][6]rel_plan size: 8
#RP_fluents 115
{4/1/3}:IW(1) -> [2][3][4][5][6][7][8][9]rel_plan size: 0
#RP_fluents 0Plan found with cost: 19
Total time: 51.521
Nodes generated during search: 43540
Nodes expanded during search: 26981
IW search completed
45 (and  (ontop book_435 plate_274) (inside hanger_380 nightstand_106) (grabbed character_1 hanger_148) (grabbed character_2 clothespile_414))


task contains conditional effects. turning off state domination.

 --- OK.
 Match tree built with 2988 nodes.

PDDL problem description loaded: 
	Domain: VIRTUALHOME
	Problem: RELAX_WATCH_TV
	#Actions: 2988
	#Fluents: 2560
Landmarks found: 4
Starting search with IW (time budget is 60 secs)...
rel_plan size: 25
#RP_fluents 296
Caption
{#goals, #UNnachieved,  #Achieved} -> IW(max_w)

{4/4/0}:IW(1) -> [2][3][4][5]rel_plan size: 21
#RP_fluents 271
{4/3/1}:IW(1) -> [2][3][4][5][6]rel_plan size: 14
#RP_fluents 230
{4/2/2}:IW(1) -> [2][3][4][5][6][7]rel_pl

In [18]:
method

{'(and  (ontop bananas_316 nightstand_106) (inside clothespants_141 clothespile_150) (grabbed character_1 hanger_119) (grabbed character_2 clothespants_379))': ['(WALK BEDROOM_74 CHARACTER_1)\n',
  '(WALK HANGER_119 CHARACTER_1)\n',
  '(GRAB HANGER_119 CHARACTER_1)\n',
  '(WALK BATHROOM_11 CHARACTER_2)\n',
  '(WALK LIVINGROOM_336 CHARACTER_2)\n',
  '(WALK CLOSET_375 CHARACTER_2)\n',
  '(OPEN CLOSET_375 CHARACTER_2)\n',
  '(WALK CLOTHESPANTS_379 CHARACTER_2)\n',
  '(GRAB CLOTHESPANTS_379 CHARACTER_2)\n',
  '(WALK KITCHEN_207 CHARACTER_2)\n',
  '(WALK BANANAS_316 CHARACTER_2)\n',
  '(GRAB BANANAS_316 CHARACTER_2)\n',
  '(WALK BEDROOM_74 CHARACTER_2)\n',
  '(WALK NIGHTSTAND_106 CHARACTER_2)\n',
  '(PUT BANANAS_316 NIGHTSTAND_106 CHARACTER_2)\n',
  '(WALK CLOSET_129 CHARACTER_2)\n',
  '(OPEN CLOSET_129 CHARACTER_2)\n',
  '(WALK CLOTHESPILE_150 CHARACTER_2)\n',
  '(WALK CLOTHESPANTS_141 CHARACTER_2)\n',
  '(GRAB CLOTHESPANTS_141 CHARACTER_2)\n',
  '(OPEN CLOTHESPILE_150 CHARACTER_2)\n',
  '

In [19]:
with open("method.json", "w") as outfile:
    json.dump(method, outfile)

## NOT NEEDED

In [ ]:
domain_name='./domain.pddl'
problem_name='./sample_env/out_problems/file_0.pddl'
output_dir = "./example_out"

In [ ]:
! ./LAPKT-public/planners/siw_plus-then-bfs_f-ffparser/siw-then-bfsf --domain $domain_name --problem $problem_name --output $output_dir



task contains conditional effects. turning off state domination.

 --- OK.
 Match tree built with 2006 nodes.

PDDL problem description loaded: 
	Domain: VIRTUALHOME
	Problem: RELAX_WATCH_TV
	#Actions: 2006
	#Fluents: 1802
Landmarks found: 4
Starting search with IW (time budget is 60 secs)...
rel_plan size: 16
#RP_fluents 193
Caption
{#goals, #UNnachieved,  #Achieved} -> IW(max_w)

{4/4/0}:IW(1) -> [2][3][4]rel_plan size: 15
#RP_fluents 220
{4/3/1}:IW(1) -> [2][3][4]rel_plan size: 11
#RP_fluents 126
{4/2/2}:IW(1) -> [2][3][4][5][6]rel_plan size: 7
#RP_fluents 105
{4/1/3}:IW(1) -> [2][3][4][5][6][7][8]rel_plan size: 0
#RP_fluents 0Plan found with cost: 18
Total time: 18.3431
Nodes generated during search: 30186
Nodes expanded during search: 17581
IW search completed


In [ ]:
f = open('./example_out')
for line in f:
    print(line)

(WALK KITCHEN_207 CHARACTER_2)

(WALK MUG_196 CHARACTER_2)

(GRAB MUG_196 CHARACTER_2)

(WALK LIVINGROOM_336 CHARACTER_1)

(WALK CLOTHESPILE_413 CHARACTER_1)

(GRAB CLOTHESPILE_413 CHARACTER_1)

(OPEN CLOTHESPILE_413 CHARACTER_1)

(WALK KITCHEN_207 CHARACTER_1)

(WALK CEREAL_335 CHARACTER_1)

(GRAB CEREAL_335 CHARACTER_1)

(PUTIN CEREAL_335 CLOTHESPILE_413 CHARACTER_1)

(WALK BATHROOM_11 CHARACTER_2)

(WALK TOWEL_68 CHARACTER_2)

(GRAB TOWEL_68 CHARACTER_2)

(WALK LIVINGROOM_336 CHARACTER_2)

(WALK CLOTHESPILE_402 CHARACTER_2)

(OPEN CLOTHESPILE_402 CHARACTER_2)

(PUTIN TOWEL_68 CLOTHESPILE_402 CHARACTER_2)



## Parse Generated Data

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
gdrive_root = "/content/drive"
data_dir = os.path.join(gdrive_root, 'MyDrive' ,'Dataset', 'lm_vh')
train_dir = os.path.join(data_dir, 'gather_data_actiongraph_train.json')
test_dir = os.path.join(data_dir, 'gather_data_actiongraph_test.json')
assert os.path.exists(data_dir), 'Check the path to your root directory'
assert os.path.exists(train_dir), 'Check the path to your root directory'
assert os.path.exists(test_dir), 'Check the path to your root directory'

In [ ]:
!pip install ijson

In [ ]:
import json
import ijson

#For training data
res = []
with open(train_dir, "r") as f:
    for record in ijson.items(f, "train_data.item"):
        res += [{'env': record['env_id'], 'goal': record['goal'], 'script':record['valid_action_with_walk']}]
        print(record['name'])

parsed_train_dir = os.path.join(data_dir, 'parsed_train.json')
with open(parsed_train_dir, 'w') as fp:
    json.dump(res, fp)

In [ ]:
#for test data
res = []
with open(test_dir, "r") as f:
    for record in ijson.items(f, "test_data.item"):
        res += [{'env': record['env_id'], 'goal': record['goal'], 'script':record['valid_action_with_walk']}]
        print(record['name'])
        
parsed_test_dir = os.path.join(data_dir, 'parsed_test.json')
with open(parsed_test_dir, 'w') as fp:
    json.dump(res, fp)